In [16]:
import pandas as pd
import numpy as np
import os

import pickle

from pathlib import Path
from music21 import converter, instrument, note, chord

from keras.utils.np_utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,LSTM,SimpleRNN, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint

In [ ]:
#pip install music21

#### Set working directory

In [17]:
path = "D:\Music generation project"

os.chdir(path)

#### Read in the midi files. 

In [ ]:
songs = []
folder = Path("D:\\Music generation project\\Data\\maestro-v3.0.0")
for file in folder.rglob('*.midi'):
  songs.append(file)

In [ ]:
#songs

In [ ]:
test_songs= songs[:10]

In [ ]:
test_songs

In [ ]:
# will need to check indentation structure and see if i've told this to do the right things, is the entire append activity only under the 'else' clause?

In [ ]:
from music21 import converter, instrument, note, chord
notes = []
for i,file in enumerate(songs):
    print(f'{i+1}: {file}')
    try:
      midi = converter.parse(file)
      notes_to_parse = None
      parts = instrument.partitionByInstrument(midi)
      if parts: # file has instrument parts
          notes_to_parse = parts.parts[0].recurse()
      else: # file has notes in a flat structure
          notes_to_parse = midi.flat.notes      
      for element in notes_to_parse:
           if isinstance(element, note.Note):
              notes.append(str(element.pitch))
           elif isinstance(element, chord.Chord):
              notes.append('.'.join(str(n) for n in element.normalOrder))
    except:
      print(f'FAILED: {i+1}: {file}')

In [ ]:
#notes

In [ ]:

with open('music_notes.pkl', 'wb') as f:
  pickle.dump(notes, f)

#### load data for new session

In [18]:
with open('music_notes.pkl', 'rb') as f:

    notes = pickle.load(f)

In [20]:
"""
def prepare_sequences(notes, n_vocab):
    #Prepare the sequences used by the Neural Network
    sequence_length = 32

    # Get all unique pitchnames
    pitchnames = sorted(set(item for item in notes))
    numPitches = len(pitchnames)

     # Create a dictionary to map pitches to integers
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

    network_input = []
    network_output = []

    # create input sequences and the corresponding outputs
    for i in range(0, len(notes) - sequence_length, 1):
        # sequence_in is a sequence_length list containing sequence_length notes
        sequence_in = notes[i:i + sequence_length] # LD - DON'T RALLY GET WHAT i:i MEANS
        # sequence_out is the sequence_length + 1 note that comes after all the notes in
        # sequence_in. This is so the model can read sequence_length notes before predicting
        # the next one.
        sequence_out = notes[i + sequence_length]
        # network_input is the same as sequence_in but it containes the indexes from the notes
        # because the model is only fed the indexes.
        network_input.append([note_to_int[char] for char in sequence_in])
        # network_output containes the index of the sequence_out
        network_output.append(note_to_int[sequence_out])

    # n_patters is the length of the times it was iterated 
    # for example if i = 3, then n_patterns = 3
    # because network_input is a list of lists
    n_patterns = len(network_input)

    # reshape the input into a format compatible with LSTM layers
    # Reshapes it into a n_patterns by sequence_length matrix
    print(len(network_input))
    
    network_input = np.reshape(network_input, (n_patterns, sequence_length, 1))
    # normalize input
    network_input = network_input / float(n_vocab)

    # OneHot encodes the network_output
    network_output = to_categorical(network_output, numPitches)

    return (network_input, network_output)
"""

"\ndef prepare_sequences(notes, n_vocab):\n    #Prepare the sequences used by the Neural Network\n    sequence_length = 32\n\n    # Get all unique pitchnames\n    pitchnames = sorted(set(item for item in notes))\n    numPitches = len(pitchnames)\n\n     # Create a dictionary to map pitches to integers\n    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))\n\n    network_input = []\n    network_output = []\n\n    # create input sequences and the corresponding outputs\n    for i in range(0, len(notes) - sequence_length, 1):\n        # sequence_in is a sequence_length list containing sequence_length notes\n        sequence_in = notes[i:i + sequence_length] # LD - DON'T RALLY GET WHAT i:i MEANS\n        # sequence_out is the sequence_length + 1 note that comes after all the notes in\n        # sequence_in. This is so the model can read sequence_length notes before predicting\n        # the next one.\n        sequence_out = notes[i + sequence_length]\n        # ne

In [ ]:
#n_vocab = len(set(notes))
#network_input, network_output = prepare_sequences(notes,n_vocab)
#n_patterns = len(network_input)
#pitchnames = sorted(set(item for item in notes))
#numPitches = len(pitchnames)

In [44]:
# First go
"""
def prepare_sequences(notes, n_vocab, sequence_length=32, batch_size=128):
    #Prepare sequences used by the Neural Network 
    pitchnames = sorted(set(notes))
    num_pitches = len(pitchnames)
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

    def generator():
        for i in range(0, len(notes) - sequence_length, batch_size):
            batch_input = []
            batch_output = []
            for j in range(batch_size):
                if i + j + sequence_length < len(notes):
                 sequence_in = notes[i + j:i + j + sequence_length]
                 sequence_out = notes[i + j + sequence_length]
                 batch_input.append([note_to_int[char] for char in sequence_in])
                 batch_output.append(note_to_int[sequence_out])
                yield np.array(batch_input), to_categorical(batch_output, num_pitches)
                
            else:
                pass 

    return generator()
"""

'\ndef prepare_sequences(notes, n_vocab, sequence_length=32, batch_size=128):\n    #Prepare sequences used by the Neural Network \n    pitchnames = sorted(set(notes))\n    num_pitches = len(pitchnames)\n    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))\n\n    def generator():\n        for i in range(0, len(notes) - sequence_length, batch_size):\n            batch_input = []\n            batch_output = []\n            for j in range(batch_size):\n                if i + j + sequence_length < len(notes):\n                 sequence_in = notes[i + j:i + j + sequence_length]\n                 sequence_out = notes[i + j + sequence_length]\n                 batch_input.append([note_to_int[char] for char in sequence_in])\n                 batch_output.append(note_to_int[sequence_out])\n                yield np.array(batch_input), to_categorical(batch_output, num_pitches)\n                \n            else:\n                pass \n\n    return generator()\n'

In [48]:
# Second go

def generator():
    try:
        for i in range(0, len(notes) - sequence_length, batch_size):
            batch_input = []
            batch_output = []
            for j in range(batch_size):
                if i + j + sequence_length < len(notes):
                    sequence_in = notes[i + j:i + j + sequence_length]
                    sequence_out = notes[i + j + sequence_length]
                    batch_input.append([note_to_int[char] for char in sequence_in])
                    batch_output.append(note_to_int[sequence_out])
            yield np.array(batch_input), to_categorical(batch_output, num_pitches)
    except Exception as e:
        print("An error occurred in the generator:", e)
        # Optionally, you can handle the error further or raise it again to propagate it

# Create and return an instance of the generator
    return generator()

In [64]:
# Usage:
n_vocab = len(set(notes))
sequence_generator = prepare_sequences(notes, n_vocab)

In [ ]:
len(notes)

In [50]:
sequence_generator

<generator object prepare_sequences.<locals>.generator at 0x0000023D00070EB0>

In [51]:
# Add a callback to save weight when the loss function declines
num_epochs = 10

filepath = "weights-improvement-{epoch:02d}-{loss:.4f}-bigger_1.hdf5"
checkpoint = ModelCheckpoint(
    filepath, monitor='loss', 
    verbose=1,        
    save_best_only=True,        
    mode='min'
)    
callbacks_list = [checkpoint]

#history = model.fit(networkInputShaped, networkOutputShaped, epochs=num_epochs, batch_size=64, callbacks=callbacks_list)

In [52]:
model = Sequential()
model.add(LSTM(256, input_shape=(32, 1)))
model.add(Dense(n_vocab, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

# Train the model using the generator function
model.fit(sequence_generator, steps_per_epoch=len(notes)//128, epochs=10, callbacks=callbacks_list) #128 is batch size

Epoch 1/10
28821/28821 [==============================] - ETA: 0s - loss: 3.7756
Epoch 1: loss improved from inf to 3.77560, saving model to weights-improvement-01-3.7756-bigger_1.hdf5
28821/28821 [==============================] - 2473s 86ms/step - loss: 3.7756
Epoch 2/10
28821/28821 [==============================] - ETA: 0s - loss: 3.8623
Epoch 2: loss did not improve from 3.77560
28821/28821 [==============================] - 2400s 83ms/step - loss: 3.8623
Epoch 3/10
28821/28821 [==============================] - ETA: 0s - loss: 3.9264
Epoch 3: loss did not improve from 3.77560
28821/28821 [==============================] - 2343s 81ms/step - loss: 3.9264
Epoch 4/10
28821/28821 [==============================] - ETA: 0s - loss: 3.9170
Epoch 4: loss did not improve from 3.77560
28821/28821 [==============================] - 2277s 79ms/step - loss: 3.9170
Epoch 5/10
28821/28821 [==============================] - ETA: 0s - loss: 3.7799
Epoch 5: loss did not improve from 3.77560
28821/2

In [ ]:
# This is quite a badly defined model, no activation function in may of these, no model.fit, etc
"""
model = Sequential()
model.add(Dropout(0.2))
model.add(LSTM(
    512,
    input_shape=(32, 1),
    return_sequences=True
))
model.add(Dense(256))
model.add(Dense(256))
model.add(LSTM(512, return_sequences=True))
model.add(Dense(256))
model.add(LSTM(512))
#model.add(Dense(numPitches))
model.add(Dense(numPitches))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
"""

In [ ]:
# DON'T APPEAR TO HAVE ANY MODEL.FIT

In [65]:
network_input_length = sum(1 for _ in sequence_generator)

In [66]:
print(network_input_length)

3689216


In [55]:
#V1
"""

def generate_notes(model, network_input, pitchnames, n_vocab):
   #Generate notes from the neural network based on a sequence of notes 
    # pick a random sequence from the input as a starting point for the prediction
    # Selects a random row from the network_input
    network_input_length = sum(1 for _ in network_input)
    
    #start = np.random.randint(0, len(network_input_length)-1)
    start = np.random.randint(0, network_input_length - 1)
    print(f'start: {start}')
    int_to_note = dict((number, note) for number, note in enumerate(pitchnames))

    # Random row from network_input
    pattern = network_input[start]
    prediction_output = []

    # generate 500 notes
    for note_index in range(500):
        # Reshapes pattern into a vector
        prediction_input = np.reshape(pattern, (1, len(pattern), 1))
        # Standarizes pattern
        prediction_input = prediction_input / float(n_vocab)

        # Predicts the next note
        prediction = model.predict(prediction_input, verbose=0)

        # Outputs a OneHot encoded vector, so this picks the columns
        # with the highest probability
        index = np.argmax(prediction)
        # Maps the note to its respective index
        result = int_to_note[index]
        # Appends the note to the prediction_output
        prediction_output.append(result)

        # Adds the predicted note to the pattern
        pattern = np.append(pattern,result)
        # Slices the array so that it contains the predicted note
        # eliminating the first from the array, so the model can
        # have a sequence
        pattern = pattern[1:len(pattern)]

    return prediction_output
    
"""

In [72]:
sequence_generator = prepare_sequences(notes, n_vocab)
    

In [68]:
# V2

def generate_notes(model, network_input_generator, pitchnames, n_vocab):
    """ Generate notes from the neural network based on a sequence of notes """
    # Count the number of elements in the generator without consuming it
    network_input_length = sum(1 for _ in network_input_generator)

    # Reset the generator to its initial state
    #network_input_generator = prepare_sequences(notes, n_vocab)
    
    # Select a random start index within the valid range
    start = np.random.randint(0, network_input_length - 1)
    print(f'start: {start}')
    int_to_note = dict((number, note) for number, note in enumerate(pitchnames))

    # Random row from network_input
    for i, (batch_input, _) in enumerate(network_input_generator):
        if i == start:
            pattern = batch_input
            break

    prediction_output = []

    # generate 500 notes
    for note_index in range(500):
        # Reshapes pattern into a vector
        prediction_input = np.reshape(pattern, (1, len(pattern), 1))
        # Standarizes pattern
        prediction_input = prediction_input / float(n_vocab)

        # Predicts the next note
        prediction = model.predict(prediction_input, verbose=0)

        # Outputs a OneHot encoded vector, so this picks the columns
        # with the highest probability
        index = np.argmax(prediction)
        # Maps the note to its respective index
        result = int_to_note[index]
        # Appends the note to the prediction_output
        prediction_output.append(result)

        # Adds the predicted note to the pattern
        pattern = np.append(pattern, index)
        # Slices the array so that it contains the predicted note
        # eliminating the first from the array, so the model can
        # have a sequence
        pattern = pattern[1:]

    return prediction_output

In [74]:
def generate_notes(model, network_input_generator, pitchnames, n_vocab):
    """ Generate notes from the neural network based on a sequence of notes """
    # Count the number of elements in the generator without consuming it
    network_input_length = sum(1 for _ in network_input_generator)

    # Reset the generator to its initial state - does this line need commenting out?
    network_input_generator = prepare_sequences(notes, n_vocab)
    
    # Select a random start index within the valid range
    start = np.random.randint(0, network_input_length - 1)
    print(f'start: {start}')
    int_to_note = dict((number, note) for number, note in enumerate(pitchnames))

    # Initialize pattern with a default value
    pattern = None

    # Random row from network_input
    for i, (batch_input, _) in enumerate(network_input_generator):
        if i == start:
            pattern = batch_input
            break

    # Check if pattern is still None (i.e., network_input_generator is empty)
    if pattern is None:
        raise ValueError("Network input generator is empty. Please check your input data.")

    prediction_output = []

    # generate 500 notes
    for note_index in range(500):
        # Reshapes pattern into a vector
        prediction_input = np.reshape(pattern, (1, len(pattern), 1))
        # Standardizes pattern
        prediction_input = prediction_input / float(n_vocab)

        # Predicts the next note
        prediction = model.predict(prediction_input, verbose=0)

        # Outputs a OneHot encoded vector, so this picks the columns
        # with the highest probability
        index = np.argmax(prediction)
        # Maps the note to its respective index
        result = int_to_note[index]
        # Appends the note to the prediction_output
        prediction_output.append(result)

        # Adds the predicted note to the pattern
        pattern = np.append(pattern, index)
        # Slices the array so that it contains the predicted note
        # eliminating the first from the array, so the model can
        # have a sequence
        pattern = pattern[1:]

    return prediction_output


In [75]:
n_vocab = len(set(notes))
pitchnames = sorted(set(item for item in notes))
prediction_output = generate_notes(model, sequence_generator, pitchnames, n_vocab)

# 'network_input' should be sequence generator

start: 3355299


ValueError: cannot reshape array of size 1152 into shape (1,36,1)

In [ ]:
CRASH

In [ ]:
def create_midi(prediction_output):
    offset = 0
    output_notes = []

    # create note and chord objects based on the values generated by the model
    for pattern in prediction_output:
        # pattern is a chord
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                new_note = note.Note(int(current_note))
                new_note.storedInstrument = instrument.Piano()
                notes.append(new_note)
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
        # pattern is a note
        else:
            new_note = note.Note(pattern)
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            output_notes.append(new_note)

        # increase offset each iteration so that notes do not stack
        offset += 0.5
    midi_stream = stream.Stream(output_notes)
    midi_stream.write('midi', fp='output.mid')